# [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial)


### 사용되는 파일

* **labeledTrainData** - 분류된 Training Set. Tab구분자로 되어 있고 컬럼의 헤더를 가진다. ID, Sentiment, Review를 25000건 포함
* **testData** - Test Set. Tab구분자이고 25000건의 리뷰 정보를 담고 있음 
* **unlabeledTrainData** - 분류되지 않은 확장 Training Set. Id와 Review 50000건을 포함 (Sentiment 없음)
* **sampleSubmission** - kaggle 제출용. Comma 구분자로 되어 있음

### Data fields
* **id** - 각 리뷰의 Unique한 ID
* **sentiment** - 리뷰의 정서(평점), 좋은 리뷰는 1 / 나쁜 리뷰는 0
* **review** - Review의 텍스트

## Reading the Data



In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "input/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "input/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "input/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



## Preprocessing
Tutorial 1과 마찬가지로 Clean Text 작업을 진행

다만 Word2Vec에서는 stop words를 제거하지 않는 것이 더 좋다. 그 이유는 알고리즘은 좋은 품질의 word vectors를 생산하기 위해 넓은 전후사정(문맥)을 필요로한다.

In [2]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

# remove_stopwords option은 기본으로 False로 한다
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

다음으로 입력 포맷을 맞춰야한다. Word2Vec은 하나의 문장이 들어오는 것을 기대한다. 

영어(한국어도)의 문장은 "?", "!", """, ".'로 끝난다. (공백과 대문자로 구분하는 것은 신뢰할 수 없음) 

NLTK의 punkt tokenizer를 이용하여 문장을 제거한다.(NLTK의 punkt 를 다운필요) 

In [3]:
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# 위의 함수에서 tokenizer를 인자로 받도록 변경.
# ReDefine a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

(오래걸림 주의)

In [4]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/user/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a U

Parsing sentences from unlabeled set


/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like r

In [5]:
print(len(sentences))

795538


In [6]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


## Training and Saving Your Model

_**word2vec option**_ (일부)

* **Architecture** - skip-gram(default) / continuous 선택 가능. skip-gram 약간 느리지만 더 좋은 결과가 나옴
* **Training algorithm** - Hierarchical softmax (Default) / Negative sampling. 기본이 더 좋은 결과가 나옴
* **Downsampling of frequent words:** - 구글 문서에서는 .00001과 .001 사이의 값을 추천한다.  마지막 모델에서 0.001일 때 정확도가 가장 증가했다.
* **Word vector dimensionality** - 많은 Feature 결과는 더 많은 시간이 든다. 
* **Context / window size:** - How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
* **Worker threads** - 병렬 프로세스의 숫자. This is computer-specific, but between 4 and 6 should work on most systems.
* **Minimum word count** - 이것은 단어의 크기를 의미 있는 단어로 제한하는 데 도움을 준다. 10~100 사이의 숫자가 합리적. 영화 제목에 너무 중요성을 두는 것을 피하기 위해  우리는 최소 가운트를 40으로 설정했다. 총 단어 크기가 15,000단어 정도가 되는 결과를 가져왔다. 값이 높을수록 실행 시간을 제한하는 데 도움이 됩니다.

[API Documentation](http://radimrehurek.com/gensim/models/word2vec.html) 

[Google Documentation](https://code.google.com/archive/p/word2vec/)



In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.word2vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-02-03 13:33:27,074 : INFO : 'pattern' package not found; tag filters are not available for English
2018-02-03 13:33:27,081 : INFO : collecting all words and their counts
2018-02-03 13:33:27,083 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-03 13:33:27,161 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-02-03 13:33:27,231 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2018-02-03 13:33:27,303 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-02-03 13:33:27,380 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-02-03 13:33:27,448 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-02-03 13:33:27,515 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-02-03 13:33:27,585 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-02-03 13:33:27,650 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-02-03 13:33:27,721 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-02-03 13:33:27,789 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-02-03 13:33:27,860 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2018-02-03 13:33:32,324 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-02-03 13:33:32,387 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-02-03 13:33:32,455 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-02-03 13:33:32,520 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-02-03 13:33:32,584 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-02-03 13:33:32,624 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-02-03 13:33:32,626 : INFO : Loading a fresh vocabulary
2018-02-03 13:33:32,726 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-02-03 13:33:32,727 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-02-03 13:3

2018-02-03 13:34:25,232 : INFO : EPOCH 3 - PROGRESS: at 53.93% examples, 618049 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:26,233 : INFO : EPOCH 3 - PROGRESS: at 58.74% examples, 618354 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:27,245 : INFO : EPOCH 3 - PROGRESS: at 63.73% examples, 619152 words/s, in_qsize 8, out_qsize 0
2018-02-03 13:34:28,252 : INFO : EPOCH 3 - PROGRESS: at 68.55% examples, 618567 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:29,252 : INFO : EPOCH 3 - PROGRESS: at 73.39% examples, 618832 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:30,259 : INFO : EPOCH 3 - PROGRESS: at 78.49% examples, 620536 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:31,280 : INFO : EPOCH 3 - PROGRESS: at 83.62% examples, 621523 words/s, in_qsize 7, out_qsize 0
2018-02-03 13:34:32,291 : INFO : EPOCH 3 - PROGRESS: at 88.58% examples, 621936 words/s, in_qsize 8, out_qsize 0
2018-02-03 13:34:33,300 : INFO : EPOCH 3 - PROGRESS: at 93.51% examples, 622024 words/s, in_qsiz

## Exploring the Model Results

self.doesnt_match는 리스트에서 가장 유사성이 없는 한 단어를 추정한다

* Tutorial에서 사용된 이 메소드는 4.0.0에서 삭제될 예정 - Deprecated
* self.wv.doesnt_match를 사용하였음

self.mv.most_similar(word) 는 가장 word와 유사한 단어들를 알려준다 

In [16]:
#model.doesnt_match("man woman child kitchen".split())

model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [19]:
# 이 기능은 완벽하지 않다. 순서에 따라 다른 결과 값을 가져오기도 하는듯.
print(model.wv.doesnt_match("france england germany berlin".split()))
print(model.wv.doesnt_match("paris berlin london austria".split()))

berlin
paris


In [22]:
print(model.wv.most_similar("man"))
print(model.wv.most_similar("queen"))
print(model.wv.most_similar("man"))

[('woman', 0.6187256574630737), ('lady', 0.5952473282814026), ('lad', 0.5658622980117798), ('monk', 0.5415423512458801), ('guy', 0.51703941822052), ('farmer', 0.5104525685310364), ('millionaire', 0.5098761320114136), ('businessman', 0.508300244808197), ('person', 0.5068169236183167), ('soldier', 0.5051417946815491)]
[('princess', 0.6983469724655151), ('bride', 0.6543586850166321), ('stepmother', 0.636001706123352), ('mistress', 0.6315175294876099), ('eva', 0.613583505153656), ('duchess', 0.6083515882492065), ('victoria', 0.6081218719482422), ('latifah', 0.6049017906188965), ('maria', 0.6001386046409607), ('regina', 0.5974386930465698)]
[('woman', 0.6187256574630737), ('lady', 0.5952473282814026), ('lad', 0.5658622980117798), ('monk', 0.5415423512458801), ('guy', 0.51703941822052), ('farmer', 0.5104525685310364), ('millionaire', 0.5098761320114136), ('businessman', 0.508300244808197), ('person', 0.5068169236183167), ('soldier', 0.5051417946815491)]


In [25]:
print(model.wv.most_similar("awful"))

[('terrible', 0.7589613199234009), ('horrible', 0.7428545355796814), ('dreadful', 0.7264014482498169), ('abysmal', 0.7125990390777588), ('atrocious', 0.710379421710968), ('horrendous', 0.6810494661331177), ('horrid', 0.6743164658546448), ('appalling', 0.6556950807571411), ('laughable', 0.6128154397010803), ('lousy', 0.6054826974868774)]
